In [ ]:
import pandas as pd, numpy as np, xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
config_path = Path(params["config_path"])
config = config_adapter.load(config_path)
RenderJSON(config)

In [ ]:
events = pd.read_csv(params["fiber_event_path"], sep=",").rename(columns=dict(Name="channel_name")).reset_index(names="original_index").sort_values(["TimeStamp", "original_index"])
events.insert(0, "t", events.pop("TimeStamp")/1000)
events

In [ ]:
res= []
for n, grp in events.groupby("channel_name"):
    # print(grp)
    if (grp["State"]!=(np.arange(len(grp.index)) % 2)).any():
        raise Exception(f"State column problem for input {n}")
    if len(grp.index) % 2 ==0:
      end = grp["t"].iloc[1::2].to_numpy()
    else:
      end=np.pad(grp["t"].iloc[1::2].to_numpy(), ((0, 1),), constant_values=np.nan)
    new_df = pd.DataFrame(dict(
       channel_name=n, 
       t=grp["t"].iloc[::2].to_numpy(),
       end=end,
       original_index=grp["original_index"].iloc[::2].to_numpy(),
    ))
    res.append(new_df)
    
events_df = pd.concat(res).sort_values(["t", "original_index"])
events_df["duration"] = events_df["end"] - events_df["t"]
events_df = events_df.drop(columns="original_index").reset_index(drop=True)
events_df

In [ ]:
cols = []
for d in config["new_events"]:
    if d["name"] in events_df.columns:
        raise Exception(f'Name {d["name"]} already in use')
    events_df[d["name"]] = events_df.eval(d["filter"], engine="python")
    cols.append(d["name"])
original_cols = [c for c in events_df.columns if not c in cols]
events_df

In [ ]:
events_df["n_used"] = events_df[cols].sum(axis=1)
events_df

In [ ]:
if config["on_unused"] =="error" and (events_df["n_used"] ==0).any():
    raise Exception("Some events where unused")
if config["on_multiple"] =="error" and (events_df["n_used"] > 1).any():
    raise Exception("Some events where used several times")
    

In [ ]:
res_df = []
for col in cols:
    res_df.append(events_df[original_cols].loc[events_df[col]].assign(event=col))
res_df = pd.concat(res_df).sort_values("t").reset_index(drop=True)[["event"]+original_cols].drop(columns="channel_name")
res_df

In [ ]:
res_df.to_csv("events.tsv", sep="\t", index=False)